### Working with the new langchain LCEL structuring tool

LCEL is an expressive language for building langchain based chains, it is a more intuitive way than the class based chain composition strategy currently used.

In [3]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

In [4]:
# load in the environment variables
load_dotenv()

True

In [5]:
mistral7b = ChatOpenAI(model="mistralai/Mistral-7B-Instruct-v0.2", temperature=0.1)
mistral7b._default_params

{'model': 'mistralai/Mistral-7B-Instruct-v0.2',
 'stream': False,
 'n': 1,
 'temperature': 0.1}

In [6]:
prompt_template = ChatPromptTemplate.from_template(
     "tell me a short joke about {topic}"
)

In [7]:
output_parser = StrOutputParser()

In [8]:
chain = prompt_template | mistral7b | output_parser

In [9]:
chain.invoke({"topic": "marvel movies"})

'Why did Iron Man skip school?\n\nBecause he had an F.F.H. (Feige-forced home)! (Referencing the Marvel movie "Iron Man" and the character Fury)'

In [10]:
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.docarray import DocArrayInMemorySearch

In [11]:
embedding = OpenAIEmbeddings(model="togethercomputer/m2-bert-80M-2k-retrieval")

In [12]:
embedding.embed_query("sample")

[-0.06494001617501273,
 0.055694497803093344,
 0.04106534162839288,
 -0.022555215824534756,
 0.030528833589660756,
 0.016337368888370712,
 0.014493061704968113,
 0.019287031670572285,
 0.01995151739832263,
 0.004259298375547931,
 -0.03045019909838228,
 -0.06010896280068027,
 -0.05079448114916055,
 -0.04705704639274952,
 -0.03357092730436969,
 -0.022228669345724053,
 0.034264052678643166,
 0.0308810717164865,
 -0.012112946077094025,
 -0.021899268239356128,
 0.028144649259514286,
 -0.01195498059771441,
 -0.0016108171215284772,
 0.020782079765189542,
 0.0006931206047843683,
 -0.017012681861112468,
 0.05018192228778666,
 0.07322806086952728,
 -0.027358402764758816,
 0.00760844498571245,
 0.016082772802526225,
 -0.027311719812866012,
 0.03895676563334412,
 -0.015101248029948425,
 -0.005331546149294046,
 0.035153860175602916,
 0.02489786370865848,
 0.03485103042301952,
 0.05814953302017113,
 0.060861121327752725,
 -0.011605524672870491,
 -0.014931362613098374,
 0.019535620471321315,
 0.00663

In [13]:
vectorstore = DocArrayInMemorySearch.from_texts(
    texts= ["harrison worked at kensho", "bears like to eat honey"],
    embedding=embedding
)

/Users/kosisochukwuasuzu/Developer/ai-startups/test-demos/pdfchat/venv/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [18]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1}) # creating a retriever from the vector store

In [19]:
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho')]

In [20]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey')]

In [27]:
template = """
[INST]
You are a question answering chatbot.
You give direct answers with provided context.
Answer the question based only on the following context:
{context}

Question: {question}
Answer:
[/INST]
"""
prompt = ChatPromptTemplate.from_template(template)

In [28]:
from langchain.schema.runnable import RunnableMap

In [29]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"] # the runnable map will fetch the following inputs and put them into the contex of the chain
}) | prompt | mistral7b | output_parser

In [31]:
chain.invoke({"question": "where does harrison work?"})

'Harrison works at Kensho.'

In [32]:
challenge_prompt = """
[INST]
You are a talented poet that writes poems about {topic}.
Write three poems in a json blob, where each poem is a json blob of a title, author, and first line.
Write just the json blob.
Poem: 
[/INST]
"""

challenge_template = ChatPromptTemplate.from_template(challenge_prompt)
prompt = challenge_template.format_messages(topic="computers")

In [33]:
mistral7b.invoke(prompt)

AIMessage(content='[\n  {\n    "title": "Whirrs and Clicks",\n    "author": "AI Poet",\n    "firstLine": "In the heart of silicon, secrets unfold"\n  },\n  {\n    "title": "Binary Ballad",\n    "author": "AI Poet",\n    "firstLine": "Through the labyrinth of circuits, love flows"\n  },\n  {\n    "title": "The Dance of Data",\n    "author": "AI Poet",\n    "firstLine": "Binary stars align, in the cosmos of the hard drive"\n  }\n]')

In [34]:
import json

In [37]:
chain = challenge_template | mistral7b | output_parser| json.loads

In [38]:
chain.invoke({"topic": "computers"})

[{'title': 'Whirrs and Clicks',
  'author': 'AI Poet',
  'firstLine': 'In the heart of silicon, secrets lie hidden'},
 {'title': 'Binary Ballad',
  'author': 'AI Poet',
  'firstLine': 'Through the labyrinth of circuits, love was found'},
 {'title': 'The Dance of Data',
  'author': 'AI Poet',
  'firstLine': 'Binary stars align in cosmic memory'}]

In [39]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)

In [40]:
chain = prompt | mistral7b | output_parser

In [41]:
chain.invoke({"topic": "bears"})

"Why don't bears like to play poker in the woods?\n\nBecause they're afraid of dealing with the big paw!"

In [42]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}]) # you can batch chains together for them to run asyncrhonously

["Why don't bears like to play poker in the woods?\n\nBecause they're afraid of dealing with the big paw!",
 'Why are frogs so happy?\n\nThey eat whatever bugs them!']